<a href="https://colab.research.google.com/github/mossssssssss/PHYGN435/blob/main/Hw5code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

# ---------- Helper Functions ------------
def complex_arcsin(z):
    """
    Compute the inverse sine (arcsin) of a (possibly complex) argument z
    using the formula:

        arcsin(z) = -i * ln(i*z + sqrt(1 - z^2))

    This function works for both real and complex z.
    """
    return -1j * np.log(1j * z + np.sqrt(1 - z**2))


def complex_cos(z):
    """
    Compute the cosine of z (which may be a complex number) using Euler's formula:

        cos(z) = (e^(iz) + e^(-iz)) / 2
    """
    return 0.5 * (np.exp(1j * z) + np.exp(-1j * z))


def compute_ellipsometric_angles(rho):
    """
    Given the complex ratio rho = r_p_total/r_s_total = tan(psi)*exp(i*delta),
    compute psi and delta (in degrees) using two equivalent approaches.

    Here we first compute psi as:

         psi = arctan(|rho|)

    Then we solve for delta using:

         delta = -Im{ ln( rho / tan(psi) ) }

    Finally, delta is adjusted to lie between 0 and 360 degrees.
    """
    # Compute psi (in radians)
    psi = np.arctan(np.abs(rho))
    # Method to compute delta:
    delta = -np.imag(np.log(rho / np.tan(psi)))

    # Convert to degrees and ensure delta is in [0, 360)
    psi_deg = np.rad2deg(psi)
    delta_deg = (np.rad2deg(delta) + 360) % 360
    return psi_deg, delta_deg


# ---------- Input parameters ------------
wavelength = 633e-9          # Wavelength: 633 nm
theta1 = np.deg2rad(70)      # Incidence angle: 70° (in radians)
d = 100e-9                   # Film thickness: 100 nm

# Refractive indices:
n1 = 1.0                     # Ambient (air)
n2 = 1.457                   # Film (SiO2) [assumed real]
n3 = 3.872 - 0.037j          # Substrate (silicon) is complex

# ---------- Compute angles in each medium ------------
# First interface (ambient -> film): n1*sin(theta1) = n2*sin(theta2)
theta2 = np.arcsin(n1 * np.sin(theta1) / n2)  # This angle is real

# Second interface (film -> substrate): n2*sin(theta2) = n3*sin(theta3)
# Since n3 is complex, we use our complex_arcsin helper.
theta3_arg = n2 * np.sin(theta2) / n3
theta3 = complex_arcsin(theta3_arg)

# ---------- Helper: Fresnel coefficients ------------
def fresnel_coeff(n_i, n_j, theta_i, theta_j):
    """
    Returns the Fresnel reflection coefficients for s and p polarizations
    at an interface between media with refractive indices n_i and n_j,
    given the incident and transmitted angles theta_i and theta_j.

    Uses the helper complex_cos for computing the cosine (which naturally handles
    both real and complex angles).
    """
    cos_theta_i = complex_cos(theta_i)
    cos_theta_j = complex_cos(theta_j)

    r_s = (n_i * cos_theta_i - n_j * cos_theta_j) / (n_i * cos_theta_i + n_j * cos_theta_j)
    r_p = (n_j * cos_theta_i - n_i * cos_theta_j) / (n_j * cos_theta_i + n_i * cos_theta_j)
    return r_s, r_p

# ---------- Compute Fresnel coefficients at each interface ------------
r12_s, r12_p = fresnel_coeff(n1, n2, theta1, theta2)
r23_s, r23_p = fresnel_coeff(n2, n3, theta2, theta3)

# ---------- Compute the phase thickness in the film ------------
beta = (2 * np.pi * n2 * d * np.cos(theta2)) / wavelength
exp_term = np.exp(-2j * beta)

# ---------- Compute the overall reflection coefficients ------------
# Standard thin-film interference formula:
r_s_total = (r12_s + r23_s * exp_term) / (1 + r12_s * r23_s * exp_term)
r_p_total = (r12_p + r23_p * exp_term) / (1 + r12_p * r23_p * exp_term)

# ---------- Compute ellipsometric parameters ------------
# From the definition: ρ = r_p_total / r_s_total = tan(psi)*exp(i*delta)
rho = r_p_total / r_s_total

# Compute ψ and δ using our helper function.
psi_deg, delta_deg = compute_ellipsometric_angles(rho)

# ---------- Print the results ------------
print("Calculated ellipsometric parameters:")
print("Psi  = {:.1f} degrees".format(psi_deg))
print("Delta = {:.1f} degrees".format(delta_deg))


Calculated ellipsometric parameters:
Psi  = 41.1 degrees
Delta = 280.4 degrees
